In [1]:
#!/usr/bin/env python

# RH2_1900-2010.ipynb

In [1]:
'''
    File name: RH2_1900-2010.ipynb
    Author: Andreas Prein
    E-mail: prein@ucar.edu
    Date created: 23.07.2020
    Date last modified: 23.07.2020

    ##############################################################
    Purpos:

    1) Reads in T2 and DT2 variables from the ERA-20C

    2) Calculate RH

    3) store the data as NetCDF


'''

'\n    File name: RH2_1900-2010.ipynb\n    Author: Andreas Prein\n    E-mail: prein@ucar.edu\n    Date created: 23.07.2020\n    Date last modified: 23.07.2020\n\n    ##############################################################\n    Purpos:\n\n    1) Reads in T2 and DT2 variables from the ERA-20C\n\n    2) Calculate RH\n\n    3) store the data as NetCDF\n\n\n'

In [1]:
from dateutil import rrule
import datetime
import glob
from netCDF4 import Dataset
import sys, traceback
import dateutil.parser as dparser
import string
from pdb import set_trace as stop
import numpy as np
import numpy.ma as ma
import os
from mpl_toolkits import basemap
import pickle
import subprocess
import pandas as pd
from scipy import stats
import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
import pylab as plt
import random
import scipy.ndimage as ndimage
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap, cm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.gridspec as gridspec
from pylab import *
import string
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import shapefile
import shapely.geometry
# import descartes
import shapefile
import math
from scipy.stats.kde import gaussian_kde
from math import radians, cos, sin, asin, sqrt
from scipy import spatial
import scipy.ndimage
import matplotlib.path as mplPath
from scipy.interpolate import interp1d
import time
from math import atan2, degrees, pi
import scipy
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters
# import SkewT
import csv
import pygrib
from scipy import interpolate

In [2]:
# ________________________________________________________________________
# ________________________________________________________________________
#             USER INPUT SECTION

DataDir='/glade/campaign/mmm/c3we/prein/ERA-20C/'
SaveDir='/glade/campaign/mmm/c3we/prein/ERA-20C/RH2/daymean/'
Vars=['T2','DT2']
Plevs=['850']# ['850','500','200']
ConstantVars='/glade/campaign/mmm/c3we/prein/ERA-20C/DT2/DT2_ERA20C_1937.nc'
FinVar='RH2'

rgdTime = pd.date_range(datetime.datetime(1900, 1, 1,0), end=datetime.datetime(2010, 12, 31,23), freq='d')
YYYYall = np.unique(rgdTime.year)

In [3]:
# ________________________________________________________________________
# ________________________________________________________________________
#              READ IN THE COORDINATES
ncid=Dataset(ConstantVars, mode='r')
rgrLat=np.squeeze(ncid.variables['lat'][:])
rgrLon=np.squeeze(ncid.variables['lon'][:])
ncid.close()

In [4]:
from thermodynamics_p3 import TandDTtoRH
for yy in range(len(YYYYall)):
    print('    process '+str(YYYYall[yy]))
    T_file=DataDir+Vars[0]+'/daymean/T2_daymean_ERA20C_'+str(YYYYall[yy])+'.nc'
    DT_file=DataDir+Vars[1]+'/daymean/DT2_daymean_ERA20C_'+str(YYYYall[yy])+'.nc'
    T_Filename=T_file.split('/')[-1]
    File_fin=SaveDir+'/'+T_Filename.replace(Vars[0], FinVar)
    File=SaveDir+T_Filename.replace(Vars[0], FinVar)+'_copy'
    if os.path.exists(File_fin) == 0:
        if not os.path.exists(SaveDir):
            os.makedirs(SaveDir)
        print( '    '+T_file)
        ncid=Dataset(T_file, mode='r')
        rgrT=np.squeeze(ncid.variables[Vars[0]][:])
        time_act=np.squeeze(ncid.variables['time'][:])
        ncid.close()
        ncid=Dataset(DT_file, mode='r')
        rgrDT=np.squeeze(ncid.variables[Vars[1]][:])
        ncid.close()

        RH_vals=TandDTtoRH(rgrT,rgrDT)

        # ________________________________________________________________________
        # write the netcdf
        print( '        ----------------------')
        print( '        Save data to '+File_fin)
        root_grp = Dataset(File, 'w', format='NETCDF4')
        # dimensions
        root_grp.createDimension('time', None)
        root_grp.createDimension('rlon', rgrLon.shape[0])
        root_grp.createDimension('rlat', rgrLat.shape[0])
        # variables
        lat = root_grp.createVariable('lat', 'f4', ('rlat',))
        lon = root_grp.createVariable('lon', 'f4', ('rlon',))
        time = root_grp.createVariable('time', 'f8', ('time',))
        RH = root_grp.createVariable('RH2', 'f4', ('time','rlat','rlon',),fill_value=-99999)

        time.calendar = "proleptic_gregorian"
        time.units = "hours since 1900-1-1 00:00:00"
        time.standard_name = "time"
        time.long_name = "time"

        lon.standard_name = "longitude"
        lon.long_name = "longitude"
        lon.units = "degrees_east"

        lat.standard_name = "latitude"
        lat.long_name = "latitude"
        lat.units = "degrees_north"

        RH.standard_name = "2m relative humidity"
        RH.long_name = "2m hPa wind speed"
        RH.units = "%"

        # write data to netcdf
        lat[:]=rgrLat
        lon[:]=rgrLon
        RH[:]=RH_vals
        time[:]=time_act
        root_grp.close()

        # compress the netcdf file
        pp = subprocess.Popen("nccopy -k 4 -d 1 -s "+File+' '+File_fin, shell=True)
        pp.wait()
        subprocess.Popen("rm  "+File, shell=True)

    process 1900
    /glade/campaign/mmm/c3we/prein/ERA-20C/T2/daymean/T2_daymean_ERA20C_1900.nc
        ----------------------
        Save data to /glade/campaign/mmm/c3we/prein/ERA-20C/RH2/daymean//RH2_daymean_ERA20C_1900.nc
    process 1901
    /glade/campaign/mmm/c3we/prein/ERA-20C/T2/daymean/T2_daymean_ERA20C_1901.nc
        ----------------------
        Save data to /glade/campaign/mmm/c3we/prein/ERA-20C/RH2/daymean//RH2_daymean_ERA20C_1901.nc
    process 1902
    /glade/campaign/mmm/c3we/prein/ERA-20C/T2/daymean/T2_daymean_ERA20C_1902.nc
        ----------------------
        Save data to /glade/campaign/mmm/c3we/prein/ERA-20C/RH2/daymean//RH2_daymean_ERA20C_1902.nc
    process 1903
    /glade/campaign/mmm/c3we/prein/ERA-20C/T2/daymean/T2_daymean_ERA20C_1903.nc
        ----------------------
        Save data to /glade/campaign/mmm/c3we/prein/ERA-20C/RH2/daymean//RH2_daymean_ERA20C_1903.nc
    process 1904
    /glade/campaign/mmm/c3we/prein/ERA-20C/T2/daymean/T2_daymean_ERA20C